# PyDOT Oct 19 2020 Final Project

## Name <span style="color:blue"><i>Gao Rui</i></span>

## Email <span style="color:blue"><i>simomo.gr@gmail.com</i></span>

## Project Name <span style="color:blue"><i>Hotel Reviews</i></span>

## Problem Statement

<span style="color:blue"><i>Scrap hotel reviews such as overall review score, scores at different aspects, reviews from guests with different attributes to visualize hotel's popularity and compare with other the hotels.</i></span>

## Data Source (if any)

<span style="color:blue"><i>We first examine guest reviews for one hotel and then extend to comparison between hotels</i></span>

Data source: booking.com
1. Data source #1 : <https://www.booking.com/hotel/sg/marina-bay-sands.html#tab-reviews> 20k+ reviews
2. Data source #2 : <https://www.booking.com/hotel/sg/rasa-sentosa-resort-by-the-shangri-la.html#tab-reviews> 1700+ reviews
3. Data source #3 : <https://www.booking.com/hotel/sg/panpacificsingapore.html#tab-reviews> 5000+ reviews

# Work breakdown and files

## Data Scraping and Wrangling
1. Given one hotel, collect ratings about `['Hotel', 'URL', 'Reviews', 'Overall Score', 'Staff', 'Cleanliness', 'Comfort', 'Value for money', 'Location', 'Free WiFi']`
2. Extract customer reviews with their attributes: `['Name', 'Origin', 'RoomType', 'StayTime', 'ReviewTime', 'ReviewScore', 'Liked', 'Disliked']`
3. Extract customer reviews/attributes based on their **review type**: ['Families', 'Couples', 'Groups', 'Solo', 'Business'] (duplicates can be seen by <code>df.sort_values(by=['Name', 'Origin', 'RoomtType', 'StayTime', 'ReviewScore'])</code>
4. Save to .csv files 

## Analysis and Visualization
For demonstration purpose: Data scraped before was saved in local directory but loaded from `./data/..`. Image is saved to local directory but loaded from `./img/..`
1. Analysis of each hotel, without **review type**
2. Analysis of each hotel, with **review type**
3. Compare hotels

In [ ]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

import hotel_extract as he
import os
from datetime import date

today = date.today()
print("Today's date:", today)

In [ ]:
# list of all hotel URL
url = ['https://www.booking.com/hotel/sg/marina-bay-sands.html#tab-reviews', 
       'https://www.booking.com/hotel/sg/rasa-sentosa-resort-by-the-shangri-la.html#tab-reviews', 
       'https://www.booking.com/hotel/sg/panpacificsingapore.html#tab-reviews']

# Data Scraping and Wrangling

## Overall Ratings of All Hotels

In [ ]:
for URL in url:
    resp = requests.get(URL)
    assert resp.status_code==200, "Request get URL unsuccessful."

    soup = BeautifulSoup(resp.text)
    
    Hotel = soup.title.text.lstrip('\n').rstrip('\n').split('(')[0].split(',')[0].strip(' ')

    print('You are viewing overal reviews for: '+Hotel)

    #  `hotel_data = ['Hotel', 'URL', 'Reviews', 'Score', 'Staff', 'Cleanliness', 'Comfort', 'Value for money', 'Location', 'Free WiFi']
    hotel_data = {'Hotel':Hotel}
    hotel_data['URL'] = URL

    overall_result = soup.find_all('span', {'class': 'review-score-widget__inline'})

    hotel_data['Overal score'] = overall_result[0].find_all('span', {'class':'review-score-badge'})[0].text.lstrip('\n').rstrip('\n')

    hotel_data['Reviews'] = overall_result[0].find_all('span', {'class':'review-score-widget__subtext'})[0].text.lstrip('\n').rstrip(' reviews\n')

    ascore_result = soup.find_all('div', {'class': 'c-score-bar'})

    for each_ascore_result in ascore_result:
        name = each_ascore_result.find_all('span',{'class':'c-score-bar__title'})[0].text.replace(u'\xa0', u' ').strip(' ')
        value = each_ascore_result.find_all('span',{'class':'c-score-bar__score'})[0].text
        hotel_data[name] = value

    hotel_data['Last updated'] = today.strftime("%d/%m/%Y")

    # save all hotel_data to a single csv file
    if os.path.isfile('./hotels.csv'):
        df_hotels = pd.read_csv('./hotels.csv')
        # check if hotel rating exists, update if not otherwise replace
        if (df_hotels['Hotel']==hotel_data['Hotel']).any():
            idx = df_hotels.loc[df_hotels['Hotel']==hotel_data['Hotel']].index
            new_df = pd.DataFrame(hotel_data, index=idx)
            df_hotels.update(new_df)
        else:
            df_hotels = df_hotels.append(pd.DataFrame(hotel_data, index=[0]), ignore_index=True)
    else:
        df_hotels = pd.DataFrame(hotel_data, index=[0])

    df_hotels.to_csv('hotels.csv', index=False)
    
print(df_hotels)

## Guest Review and Attributes for One Hotel

In [ ]:
URL = url[0]
Hotel = df_hotels.loc[0,'Hotel']
print('You are scraping data from: '+Hotel)

Note: we find that customer reivews are `client-side rendered`. - Selenium

In [ ]:
cust_header = ['Name', 'Origin', 'RoomType', 'StayTime', 'ReviewTime','ReviewScore', 'Liked', 'Disliked']
# The language of the last two may not be in English

In [ ]:
# check whether hotel review document exists
os.path.isfile('./'+Hotel+'.csv') 

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

import time

# get an instance of web browser
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(URL)
# wait for 10 seconds before timeout
wait = WebDriverWait(browser, 10)
# wait until an element is present
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'review_list')))
# Received cookies and HTML
cookies = browser.get_cookies()

### Scrap Guest Review by Clicking Next Page

In [ ]:
Name = []
Origin = []
RoomType = []
StayTime = []
ReviewTime = []
ReviewScore = []
Liked = []
Disliked = []
page = 1
sleeptime = 2

while True:
    time.sleep(sleeptime)
    html = browser.page_source # restart from here at every time page next is clicked
    soup = BeautifulSoup(html)
    cust_result = soup.find_all('div', {'class': "c-review-block"})
    
    for each_cust_result in cust_result:
        roominfo = each_cust_result.find_all('div',{'class':'c-review-block__room-info__name'})
        good, bad = he.grab_comments(each_cust_result)
        
        Name.append(each_cust_result.find_all('span',{'class':'bui-avatar-block__title'})[0].text)
        
        # some origin information may be empty
        origin_result = each_cust_result.find_all('span',{'class':'bui-avatar-block__subtitle'})
        Origin.append(origin_result[0].text.lstrip('\n') if bool(origin_result) else '')
        RoomType.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[0] if bool(roominfo) else '')
        
        StayTime.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[-1] if bool(roominfo) else '')
                
        # Stay-in time comes with room type and may be unavailable for earlier guests
        ReviewTime.append(each_cust_result.find_all('span', {'class':'c-review-block__date'})[0]
                          .text.lstrip('\n').rstrip('\n').lstrip('Reviewed: '))        
        ReviewScore.append(each_cust_result.find_all('div',{'class':'c-guest-with-score__score'})[0]
                           .text.lstrip().rstrip('\n').rstrip())
        Liked.append(good)
        Disliked.append(bad)

    if page%50==0:
        print(f'Page {page} onwards...')

    try: # click next page
        button_nextpage = browser.find_element_by_xpath(
            '//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a') 
        button_nextpage.click()
        page += 1
    except:
        print(f'Tried to press next button on page {page} and stopped.')
        break

In [ ]:
len(Name), len(Origin), len(RoomType), len(StayTime), len(ReviewTime), len(ReviewScore), len(Liked), len(Disliked)

In [ ]:
df = pd.DataFrame({'Name':Name, 'Origin':Origin, 'RoomType':RoomType, 'StayTime':StayTime, 'ReviewTime':ReviewTime , 'ReviewScore':ReviewScore, 'Liked':Liked, 'Disliked':Disliked})
df.to_csv(Hotel+'.csv', index=False)

In [ ]:
# ## Check wether the page reach the ends, need to automate this process

# # if stopped before the end of page
# button_nextpage = browser.find_element_by_xpath(
#             '//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a') 
# button_nextpage.click()
# # page += 1

# while True:
#     time.sleep(sleeptime)
#     html = browser.page_source # restart from here at every time page next is clicked
#     soup = BeautifulSoup(html)
#     cust_result = soup.find_all('div', {'class': "c-review-block"})
    
#     for each_cust_result in cust_result:
#         roominfo = each_cust_result.find_all('div',{'class':'c-review-block__room-info__name'})
#         good, bad = he.grab_comments(each_cust_result)
        
#         Name.append(each_cust_result.find_all('span',{'class':'bui-avatar-block__title'})[0].text)
        
#         # some origin information may be empty
#         origin_result = each_cust_result.find_all('span',{'class':'bui-avatar-block__subtitle'})
#         Origin.append(origin_result[0].text.lstrip('\n') if bool(origin_result) else '')
#         RoomType.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[0] if bool(roominfo) else '')
        
#         StayTime.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[-1] if bool(roominfo) else '')
                
#         # we use reviewed time instead of stay-in time as stay-in time comes with room type and may be unavailable for earlier guests
#         ReviewTime.append(each_cust_result.find_all('span', {'class':'c-review-block__date'})[0].text.lstrip('\n').rstrip('\n').lstrip('Reviewed: '))        
#         ReviewScore.append(each_cust_result.find_all('div',{'class':'c-guest-with-score__score'})[0].text.lstrip().rstrip('\n').rstrip())
#         Liked.append(good)
#         Disliked.append(bad)

#     if page%50==0:
#         print(f'Page {page} onwards...')

#     try: # click next page
#         button_nextpage = browser.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a') 
#         button_nextpage.click()
#         page += 1
#     except:
#         print(f'Tried to press next button on page {page} and stopped.')
#         break

# df = pd.DataFrame({'Name':Name, 'Origin':Origin, 'RoomType':RoomType, 'StayTime':StayTime, 'ReviewTime':ReviewTime , 'ReviewScore':ReviewScore, 'Liked':Liked, 'Disliked':Disliked})
# df.to_csv(Hotel+'.csv', index=False)

In [ ]:
browser.close()

## Guest Review and Attributes based on  <span style="color:blue"><i>ReviewType</i></span>

In [ ]:
ReviewTypeAll = ['Families', 'Couples', 'Groups', 'Solo', 'Business']

In [ ]:
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.wait import WebDriverWait

# import time

# get an instance of web browser
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get(URL)
# wait for 10 seconds before timeout
wait = WebDriverWait(browser, 10)
# wait until an element is present
wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'review_list')))
# Received cookies and HTML
cookies = browser.get_cookies()

In [ ]:
rt = browser.find_element_by_id('reviewer_type_filter')

In [ ]:
cust_header = ['ReviewType', 'Name', 'Origin', 'RoomType', 'StayTime', 'ReviewTime', 'ReviewScore', 'Liked', 'Disliked']

In [ ]:
Name = []
Origin = []
RoomType = []
ReviewTime = []
StayTime = []
ReviewScore = []
Liked = []
Disliked = []
ReviewType = []

for option in rt.find_elements_by_tag_name('option'):
    if option.text.split()[0] in ReviewTypeAll: # if ReviewType is one of the All
        option.click() # select() in earlier versions of webdriver
               
        print('Collecting '+option.text.split()[-1].strip('()')+
              ' reviews from TYPE: '+option.text.split()[0])
        
        page = 1
        count = 0
        '''
        Below are the code blocks to extract
        '''
        while True:
            time.sleep(sleeptime)
            html = browser.page_source # restart from here at every time page next is clicked
            soup = BeautifulSoup(html)
            cust_result = soup.find_all('div', {'class': "c-review-block"})

            for each_cust_result in cust_result:
                ReviewType.append(option.text.split()[0])
                
                roominfo = each_cust_result.find_all('div',{'class':'c-review-block__room-info__name'})
                good, bad = he.grab_comments(each_cust_result)

                Name.append(each_cust_result.find_all('span',{'class':'bui-avatar-block__title'})[0].text)

                # some origin information may be empty
                origin_result = each_cust_result.find_all('span',{'class':'bui-avatar-block__subtitle'})
                Origin.append(origin_result[0].text.lstrip('\n') if bool(origin_result) else '')
                RoomType.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[0] if bool(roominfo) else '')

                StayTime.append(roominfo[0].text.split(':')[1].strip(' ').strip('\n').split('\n')[-1] if bool(roominfo) else '')
                
                # we use reviewed time instead of stay-in time as stay-in time comes with room type and may be unavailable for earlier guests
                ReviewTime.append(each_cust_result.find_all('span', {'class':'c-review-block__date'})[0].text.lstrip('\n').rstrip('\n').lstrip('Reviewed: '))
                ReviewScore.append(each_cust_result.find_all('div',{'class':'c-guest-with-score__score'})[0].text.lstrip().rstrip('\n').rstrip())
                Liked.append(good)
                Disliked.append(bad)
                
            count += len(cust_result)

            if page%100==0:
                print(f'Page {page} onwards...')

            try: # click next page
                button_nextpage = browser.find_element_by_xpath('//*[@id="review_list_page_container"]/div[4]/div/div[1]/div/div[3]/a') 
                button_nextpage.click()
                page += 1
            except:
                print(f'Tried to press next button on page {page} and stopped.')
                break
        print(f'Collected {count} reviews from TYPE {option.text.split()[0]}')

Actual listings are more than stated. Summing up all categories gets more than total. Duplicates appears in different Review Types

In [ ]:
len(ReviewType), len(Name), len(Origin), len(RoomType), len(StayTime), len(ReviewTime), len(ReviewScore), len(Liked), len(Disliked)

In [ ]:
df = pd.DataFrame({'ReviewType':ReviewType, 'Name':Name, 'Origin':Origin, 'RoomType':RoomType, 
                   'StayTime': StayTime, 'ReviewTime':ReviewTime, 'ReviewScore':ReviewScore, 'Liked':Liked, 'Disliked':Disliked})
df.to_csv(Hotel+'-reviewtype.csv', index=False)

In [ ]:
browser.close()

# Analysis and Visualization

In [ ]:
import numpy as np

please note that the scaped data are saved to subfolder **./data/**

## One Hotel (No <span style="color:blue"><i>ReviewType</i></span>)

In [ ]:
df = pd.read_csv('./data/'+Hotel+'.csv')

### Review Score Distribution

In [ ]:
bins = np.arange(0,11,.5)
df['ReviewScore'].plot(kind='hist', bins=bins, figsize=(20,10), title=Hotel+': Guest Review Rating Distribution')

### Ratings over Time

In [ ]:
# As old comments does not have StayTime, we use ReviewTime if StayTime is empty
df['Time'] = df['StayTime']
df['Time'] = df['Time'].fillna(df['ReviewTime'])

In [ ]:
# time string to time
df['StayTime'] = pd.to_datetime(df['StayTime'], errors='coerce')
df['ReviewTime'] = pd.to_datetime(df['ReviewTime'], errors='coerce')
df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
df.info()

In [ ]:
df.tail()

In [ ]:
df = df.set_index('Time') # set time as index

In [ ]:
df = df.sort_values(by=['Time']) # sort by time

In [ ]:
df['Year'] = df.index.year
df['Month'] = df.index.month
df['Weekday'] = df.index.weekday
df['monthyear'] = df.index.to_period('M') # display by month-year

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(16,8))
sns.boxplot(data=df, x = 'monthyear', y = 'ReviewScore')
plt.xticks(rotation=70)
plt.title(Hotel+'; Review Scores over Time')
plt.show()

In [ ]:
df['monthyear'].value_counts().sort_values().sort_index().plot(kind='bar',figsize=(16.7, 8.27)) # count how many reviews in each monthyear

### Pie chart for origin

In [ ]:
df['Origin'].value_counts()[:20].plot(kind="pie", figsize=(10,10), title=Hotel+': Top 20 countries')

In [ ]:
### Room Type Pie Chart
df['RoomType'].unique()

In [ ]:
for index, row in df.iterrows():
    if bool(row['RoomType']) and isinstance(row['RoomType'], str) :
        test = row['RoomType'].split('(')[0].strip(' ').split('-') # remove brackets behind
        if len(test)==1:
            df.loc[index,'RoomType'] = test
        elif len(test) == 2:
            df.loc[index,'RoomType'] = test[1].strip(' ') if (test[0].strip(' ')=='Special Offer' or test[0].strip(' ')=='Special' or test[0].strip(' ')=='Festive Promo') else test[0].strip(' ')
    else:
        df.loc[index,'RoomType'] = ''

In [ ]:
df['RoomType'].unique()

In [ ]:
# we need to clean room type strings
df['RoomType'].value_counts()[:10].plot(kind="pie", figsize=(10,10), title=Hotel+': Top 10 RoomType')

### `Word Cloud` - How guests comment

In [ ]:
# Liked and Disliked word cloud
from os import path
from PIL import Image
import os

from wordcloud import WordCloud, STOPWORDS
from langdetect import detect
import re

join all Liked/Disliked Reviews respectively, Choose only English

In [ ]:
LikedAll = ''
for index, row in df.iterrows():
    if isinstance(row['Liked'], str):
        resub = re.sub('\W+',' ', row['Liked']) # remove weird characters.
        if len(resub)>1 and 'en'==detect(resub):
            LikedAll = LikedAll + ' ' + row['Liked']

In [ ]:
DislikedAll = ''
for index, row in df.iterrows():
    if isinstance(row['Disliked'], str):
        resub = re.sub('\W+',' ', row['Disliked']) # remove weird characters.
        try:
            if len(resub)>1 and 'en'==detect(resub):
                DislikedAll = DislikedAll + ' ' + row['Disliked']
        except:
            print(resub)

In [ ]:
DislikedAll

In [ ]:
STOPWORDS

In [ ]:
# count the unique words and put some input stopwords if not in stopwords

In [ ]:
stopwords = set(STOPWORDS)
stopwords.add("said")
stopwords.add("hotel")
stopwords.add("day")
stopwords.add("really")
stopwords.add("much")

try:
    mbs_mask = np.array(Image.open(f'{Hotel}_mask.png'))
    wc = WordCloud(background_color="white", max_words=2000, mask=mbs_mask,
               stopwords=stopwords, contour_width=3, contour_color='steelblue')
except:
    wc = WordCloud(background_color="white", max_words=2000, stopwords=stopwords, contour_width=3)


# 1. generate word cloud for all Liked
wc.generate(LikedAll)

# store to file
wc.to_file(Hotel+'_Liked.png')

# show
plt.figure(figsize=(30,15)) 
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

# 2. generate word cloud for all Disliked
wc = WordCloud(background_color="black", max_words=2000, stopwords=stopwords, contour_width=3)


wc.generate(DislikedAll)

# store to file
wc.to_file(Hotel+'_Disliked.png')

# show
plt.figure(figsize=(20,10)) 
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.show()

## One Hotel (with Additional Category  <span style="color:blue"><i>ReviewType</i></span>)

In [ ]:
df = pd.read_csv('./data/'+Hotel+'-reviewtype.csv')

In [ ]:
# pie plot to count the portion
df['ReviewType'].value_counts().plot(kind='pie', figsize=(10,10), title=Hotel+': Review Type Counts')

In [ ]:
fig = plt.figure(figsize=(16,8))
sns.boxplot(data=df, x = 'ReviewType', y = 'ReviewScore')
# plt.xticks(rotation=70)
plt.show()

## Comparing Hotels

In [ ]:
df_hotels = pd.read_csv('./data/hotels.csv')
df_hotels = df_hotels.set_index(['Hotel'])
df_hotels

In [ ]:
ax = df_hotels.T.drop(['URL','Reviews','Last updated']).plot.bar(figsize=(14,6))

In [ ]:
# Review Type Comparison

In [ ]:
df_rt = []
df_o = []
df_rs = []
df_means = []
for hotel in df_hotels.index:
    df = pd.read_csv('./data/'+hotel+'-reviewtype.csv')
    df_rs.append(df['ReviewScore'].rename(hotel))
    df_rt.append(df['ReviewType'].value_counts().rename(hotel))
    df_o.append(df['Origin'].value_counts()[:5].rename(hotel))
    df_means.append(df.groupby('ReviewType').mean().rename(columns={'ReviewScore':hotel}, inplace=False))

In [ ]:
df_means

In [ ]:
pd.concat(df_means, axis=1).plot.bar(figsize=(14,6), title='Mean Review Score in each Review Type')

In [ ]:
pd.concat(df_rt, axis=1).plot.pie(subplots=True, figsize=(16,16))

In [ ]:
from IPython.display import Image
Image("./img/Shangri-La's Rasa Sentosa Resort & Spa_Liked.png", width= 1000)

In [ ]:
ax = pd.concat(df_rs, axis=1).plot.density(figsize=(16,8), xlim=(0,11),title='Review Score Distribution')

In [ ]:
pd.concat(df_o, axis=1).plot.pie(subplots=True, figsize=(16,16))